https://www.youtube.com/watch?v=Y28xXQmju64&t=218s

https://docs.ultralytics.com/tasks/pose/#what-are-the-available-ultralytics-yolov8-pose-models-and-their-performance-metrics

先安裝環境

In [ ]:
'''
# 在anaconda prompt 建立環境

# 以下在commandline模式中
conda create --name yolov8 python==3.10.6  

# 進入yolov8環境中 
conda activate yolov8

# 切換目錄 自訂的資料夾
D:
cd D:\yolo

# Pip install the ultralytics  Python>=3.8 environment with PyTorch>=1.8.
pip install ultralytics

# install pyTorch 安裝檔不小有幾GB
nvcc --version
conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch -c nvidia

'''

example 1 #simple detect

In [ ]:

from ultralytics import YOLO
model = YOLO('yolov8m-pose.pt')
results = model(source=0, show=True, conf=0.3, save=True) 


example 2 # add keyboard exit

In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO('yolov8m-pose.pt')

cap = cv2.VideoCapture(0)  # Open the default camera (source=0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.3)  # Perform inference
    annotated_frame = results[0].plot()  # Get the annotated frame

    cv2.imshow('YOLOv8 Pose Estimation', annotated_frame)
    cv2.imwrite('results.jpg', annotated_frame)  # Save the frame

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Check for 'q' key press
        break

cap.release()
cv2.destroyAllWindows()

Workouts Monitoring Example
https://docs.ultralytics.com/guides/workouts-monitoring/#real-world-applications

In [ ]:
import cv2

from ultralytics import YOLO, solutions

model = YOLO("yolov8n-pose.pt")
# cap = cv2.VideoCapture("path/to/video/file.mp4")
# cap = cv2.VideoCapture("video/pushup1.mp4")
cap = cv2.VideoCapture(0)  # Open the default camera (source=0)

assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

gym_object = solutions.AIGym(
    line_thickness=2,
    view_img=True,
    pose_type="pushup",
    kpts_to_check=[6, 8, 10],
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    results = model.track(im0, verbose=False)  # Tracking recommended
    # results = model.predict(im0)  # Prediction also supported
    im0 = gym_object.start_counting(im0, results)

cv2.destroyAllWindows()

In [9]:
import cv2

from ultralytics import YOLO, solutions

# Load YOLO model
model = YOLO("yolov8n-pose.pt")

# Open the default camera (source=0)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error opening video capture")
    exit()

# Get video properties
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize AI Gym object for pushup counting
gym_object = solutions.AIGym(
    line_thickness=2,
    view_img=True,
    pose_type="pushup",
    kpts_to_check=[6, 8, 10],
)

# Process video frames
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue  # Skip empty frames

    # Perform pose estimation and tracking
    results = model.track(im0, verbose=False) 

    # Count pushups using AI Gym
    im0 = gym_object.start_counting(im0, results)

    # If you want to display the processed frame, uncomment the following line
    # cv2.imshow("Pushup Counter", im0)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
# 增加功能: 按下 s 表示 webcam結束 並且記錄次數在 變數中並寫入json中 紀錄時間 次數 狀態 

In [3]:
import cv2
import json
from datetime import datetime

from ultralytics import YOLO, solutions

# Load YOLO model
model = YOLO("yolov8n-pose.pt")

# Open the default camera (source=0)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error opening video capture")
    exit()

# Get video properties
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize AI Gym object for pushup counting
gym_object = solutions.AIGym(
    line_thickness=2,
    view_img=True,
    pose_type="pushup",
    kpts_to_check=[6, 8, 10],
)

# Process video frames
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue  # Skip empty frames

    # Perform pose estimation and tracking
    results = model.track(im0, verbose=False) 

    # Count pushups using AI Gym
    im0 = gym_object.start_counting(im0, results)

    # If you want to display the processed frame, uncomment the following line
    # cv2.imshow("Pushup Counter", im0)

    # Check for 's' key press to stop and save data
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):
        # Get current time and pushup count
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        pushup_count = gym_object.count  # Assuming gym_object has a 'count' attribute

        # Create data dictionary
        data = {
            "time": current_time,
            "count": pushup_count,
            "status": "completed"  # You can customize the status as needed
        }

        # Write data to JSON file
        with open("pushup_data.json", "w") as f:
            json.dump(data, f)

        break  # Exit the loop

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()

寫出次數到json

In [52]:
!pip install pafy

In [54]:
import cv2
from ultralytics import YOLO, solutions

model = YOLO("yolov8n-pose.pt")

# cap = cv2.VideoCapture(0)  # Open the default camera (source=0)
cap = cv2.VideoCapture("video/pushup3.mp4")

assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

video_writer = cv2.VideoWriter("workouts.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

gym_object = solutions.AIGym(
    line_thickness=2,
    view_img=True,
    pose_type="pushup",
    kpts_to_check=[6, 8, 10],
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    results = model.track(im0, verbose=False)  # Tracking recommended
    # results = model.predict(im0)  # Prediction also supported
    im0 = gym_object.start_counting(im0, results)
    out_count = gym_object.count
    print(out_count)
    with open("save_data.json", "w") as f:
        json.dump(out_count, f)
    video_writer.write(im0)

cv2.destroyAllWindows()
video_writer.release()

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[3]
[3]
[3]
[3]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[5]
[5]
[5]
[5]
[5]
[5]
[5]
[5]
[5]


要自行訓練分辨pushup pullup squat

In [ ]:
import cv2
from ultralytics import YOLO, solutions

# Load models
classification_model = YOLO("exercise_classification_model.pt")
pose_models = {
    "pushup": YOLO("yolov8n-pose-pushup.pt"),
    "pullup": YOLO("yolov8n-pose-pullup.pt"),
    "squat": YOLO("yolov8n-pose-squat.pt")
}

cap = cv2.VideoCapture("video/pushup3.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

gym_objects = {
    "pushup": solutions.AIGym(line_thickness=2, view_img=True, pose_type="pushup", kpts_to_check=[6, 8, 10]),
    "pullup": solutions.AIGym(line_thickness=2, view_img=True, pose_type="pullup", kpts_to_check=[6, 8, 10]),
    "squat": solutions.AIGym(line_thickness=2, view_img=True, pose_type="squat", kpts_to_check=[6, 8, 10])
}

current_exercise = None
frame_count = 0

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    frame_count += 1

    # Classify exercise type every few frames
    if frame_count % 30 == 0:  # Adjust frequency as needed
        classification_results = classification_model.predict(im0)
        current_exercise = classification_results[0].probs.top1  # Assuming top1 gives the exercise type

    if current_exercise:
        pose_model = pose_models[current_exercise]  # current_exercise
        gym_object = gym_objects[current_exercise]  # current_exercise
        results = pose_model.track(im0, verbose=False)
        im0 = gym_object.start_counting(im0, results, frame_count)

    cv2.imshow("Workout Monitoring", im0)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

------------ basic track -------------------

In [41]:
from ultralytics import YOLO

# Configure the tracking parameters and run the tracker
model = YOLO("yolov8n.pt")
results = model.track(source="https://www.youtube.com/shorts/HHRDXEG1YCU", conf=0.3, iou=0.5, show=True)
print(results)


1/1: https://www.youtube.com/shorts/HHRDXEG1YCU... Success  (450 frames of shape 608x1080 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 640x384 1 person, 31.6ms
0: 640x384 1 person, 22.6ms
0: 640x384 1 person, 23.3ms
0: 640x384 1 person, 26.3ms
0: 640x384 1 person, 17.1ms
0: 640x384 1 person, 30.2ms
0: 640x384 1 person, 23.1ms
0: 640x384 1 person, 20.5ms
0: 640x384 1 person, 24.3ms
0: 640x384 1 person, 26.3ms
0: 640x384 1 person, 24.1ms
0: 640x384 1 person, 30.2ms
0: 640x384 1 pers